In [1]:
import os
import math
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg, fft

import time as time
import platform as platform

import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import L2
import h5py

tf.keras.backend.set_floatx('float32')
plt.rcParams.update({"text.usetex":True})

In [2]:
colab_flag = False

FTYPE = np.float32
ITYPE = np.int32

strategy = None
# strategy = tf.distribute.MirroredStrategy()

In [3]:
current_sys = platform.system()

if current_sys == 'Windows':
    dir_sep = '\\'
else:
    dir_sep = '/'

In [4]:
if colab_flag == True:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Github/MLROM/KS/')

In [5]:
print(os.getcwd())

/home/rkaushik/Documents/Thesis/MLROM/Kolmogorov


In [6]:
from tools.ae_v5 import Autoencoder as ae_org
from tools.ae_v9 import Autoencoder as ae_new

In [7]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

if colab_flag == False:
    if strategy is None:
        if gpus:
            gpu_to_use = 1
            tf.config.set_visible_devices(gpus[gpu_to_use], 'GPU')
    logical_devices = tf.config.list_logical_devices('GPU')
    print(logical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2023-06-14 03:38:13.349230: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 03:38:13.349496: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 03:38:13.383952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 03:38:13.384223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 03:38:13.384424: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [8]:
# print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())
print(tf.config.list_logical_devices())
print(tf.__version__)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
[LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU')]
2.8.1


# Loading and Converting AE

In [9]:
dir_name_all_ae = os.getcwd()+'{ds}saved_ae'.format(ds=dir_sep)

In [10]:
all_ae = os.listdir(dir_name_all_ae)
chosen_dir_list = []

for dir_name in all_ae:
    with open(dir_name_all_ae+'/'+dir_name+'/ae_data.txt', 'r') as f:
        try:
            ae_dict = eval(f.readline())
            if ae_dict['module'].endswith('ae_v5'):
                chosen_dir_list.append(dir_name)
        except:
            print(dir_name)
            continue

print(chosen_dir_list)

['ae_036', 'ae_038', 'ae_037', 'ae_031', 'ae_035']


In [11]:
for dir_name in chosen_dir_list[1:2]:
    print(dir_name)
    dir_name_ae = dir_name_all_ae + '/' + dir_name
    
    load_file = dir_name_ae+dir_sep+'final_net'+dir_sep+'final_net_class_dict.txt'
    wt_file = dir_name_ae+dir_sep+'final_net'+dir_sep+'OLD_final_net_ae_weights.h5'
    
    ae_net_new = ae_new(load_file=load_file)
    ae_net_org = ae_org(load_file=load_file)
    
    ae_net_new.load_weights(wt_file)#, by_name=True)
    ae_net_org.load_weights_from_file(wt_file)

    num_kernels = len(ae_net_new.kernel_size)
    
    if num_kernels > 1:
        encoded_vec_wts = []
        decoded_vec_wts = []
        trainable = True
        with open(dir_name_ae+'/encoded_decoded_vec_weights.txt', 'r') as f:
            lines = f.readlines()
            for line in lines[2:2+num_kernels]:
                idx = line.find('.')
                encoded_vec_wts.append(float(line[idx-1:]))
            for line in lines[num_kernels+3 + 2:]:
                idx = line.find('.')
                decoded_vec_wts.append(float(line[idx-1:]))
    else:
        encoded_vec_wts = [1.]
        decoded_vec_wts = [1.]
        trainable = False
        
                
    print('\n encoded_vec_wts : ', encoded_vec_wts)
    print('\n decoded_vec_wts : ', decoded_vec_wts)
        
    for ks_i in range(num_kernels):
        ae_net_new.create_enc_wt(encoded_vec_wts[ks_i], trainable=trainable)
    for ks_i in range(num_kernels):
        ae_net_new.create_dec_wt(decoded_vec_wts[ks_i], trainable=trainable)


    for ks_i in range(num_kernels):
        print('post_setting : ae_net_new.encoded_vec_weights[{}] : '.format(ks_i),
              ae_net_new.encoded_vec_layers[ks_i].scalar_multiplier.numpy())
    for ks_i in range(num_kernels):
        print('post_setting : ae_net_new.decoded_vec_weights[{}] : '.format(ks_i),
              ae_net_new.decoded_vec_layers[ks_i].scalar_multiplier.numpy())

    ae_net_new.update_models()
        
    ae_net_new.save_model_weights(dir_name_ae+'/final_net/final_net_ae_weights', H5=True)

#     if dir_name != 'ae_032':
#         del(ae_net_new)
#         del(ae_net_org)

ae_038
ListWrapper([5]) <class 'tensorflow.python.training.tracking.data_structures.ListWrapper'>
[5] <class 'numpy.ndarray'>
[TensorShape([50, 50]), (24, 24)]
[TensorShape([50, 50]), (24, 24), (12, 12)]
[TensorShape([50, 50]), (24, 24), (12, 12), (6, 6)]
[TensorShape([50, 50]), (24, 24), (12, 12), (6, 6), (3, 3)]
[TensorShape([50, 50]), (24, 24), (12, 12), (6, 6), (3, 3), (3, 3)]
0 -- kernelsize : 5 -- 0 / 4
(3, 3)
0 -- kernelsize : 5 -- 1 / 4
(3, 3)
0 -- kernelsize : 5 -- 2 / 4
(6, 6)
0 -- kernelsize : 5 -- 3 / 4
(12, 12)
0 -- kernelsize : 5 -- 4 / 4
(24, 24)
(59, 59)


2023-06-14 03:38:15.816954: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401


ListWrapper([5]) <class 'tensorflow.python.training.tracking.data_structures.ListWrapper'>
[5] <class 'numpy.ndarray'>
[TensorShape([50, 50]), (24, 24)]
[TensorShape([50, 50]), (24, 24), (12, 12)]
[TensorShape([50, 50]), (24, 24), (12, 12), (6, 6)]
[TensorShape([50, 50]), (24, 24), (12, 12), (6, 6), (3, 3)]
[TensorShape([50, 50]), (24, 24), (12, 12), (6, 6), (3, 3), (3, 3)]
0 -- kernelsize : 5 -- 0 / 4
(3, 3)
0 -- kernelsize : 5 -- 1 / 4
(3, 3)
0 -- kernelsize : 5 -- 2 / 4
(6, 6)
0 -- kernelsize : 5 -- 3 / 4
(12, 12)
0 -- kernelsize : 5 -- 4 / 4
(24, 24)
(59, 59)

 encoded_vec_wts :  [1.0]

 decoded_vec_wts :  [1.0]
post_setting : ae_net_new.encoded_vec_weights[0] :  [1.]
post_setting : ae_net_new.decoded_vec_weights[0] :  [1.]


In [12]:
for l in ae_net_org.layers:
    print(l.name)

gaussian_noise_1
periodic_padding_10
conv2d_5
batch_normalization_10
activation_10
periodic_padding_11
conv2d_6
batch_normalization_11
activation_11
periodic_padding_12
conv2d_7
batch_normalization_12
activation_12
periodic_padding_13
conv2d_8
batch_normalization_13
activation_13
periodic_padding_14
conv2d_9
batch_normalization_14
activation_14
model_3
periodic_padding_15
conv2d_transpose_5
cropping2d_5
batch_normalization_15
activation_15
periodic_padding_16
conv2d_transpose_6
cropping2d_6
batch_normalization_16
activation_16
periodic_padding_17
conv2d_transpose_7
cropping2d_7
batch_normalization_17
activation_17
periodic_padding_18
conv2d_transpose_8
cropping2d_8
batch_normalization_18
activation_18
periodic_padding_19
conv2d_transpose_9
cropping2d_9
batch_normalization_19
activation_19
model_4
model_5


In [13]:
for l in ae_net_org.layers:
    if l.name.startswith('model'):
        print(l.name)
        print('-'*20)
        for l2 in l.layers:
            print(l2.name)
        print('\n\n')

model_3
--------------------
input_4
gaussian_noise_1
periodic_padding_10
conv2d_5
batch_normalization_10
activation_10
periodic_padding_11
conv2d_6
batch_normalization_11
activation_11
periodic_padding_12
conv2d_7
batch_normalization_12
activation_12
periodic_padding_13
conv2d_8
batch_normalization_13
activation_13
periodic_padding_14
conv2d_9
batch_normalization_14
activation_14
tf.math.multiply_2



model_4
--------------------
input_5
periodic_padding_15
conv2d_transpose_5
cropping2d_5
batch_normalization_15
activation_15
periodic_padding_16
conv2d_transpose_6
cropping2d_6
batch_normalization_16
activation_16
periodic_padding_17
conv2d_transpose_7
cropping2d_7
batch_normalization_17
activation_17
periodic_padding_18
conv2d_transpose_8
cropping2d_8
batch_normalization_18
activation_18
periodic_padding_19
conv2d_transpose_9
cropping2d_9
batch_normalization_19
activation_19
tf.math.multiply_3



model_5
--------------------
input_6
model_3
model_4





In [14]:
for l in ae_net_new.layers:
    if l.name.startswith('model'):
        print(l.name)
        print('-'*20)
        for l2 in l.layers:
            print(l2.name)
        print('\n\n')

model
--------------------
input_7
gaussian_noise
periodic_padding
conv2d
batch_normalization
activation
periodic_padding_1
conv2d_1
batch_normalization_1
activation_1
periodic_padding_2
conv2d_2
batch_normalization_2
activation_2
periodic_padding_3
conv2d_3
batch_normalization_3
activation_3
periodic_padding_4
conv2d_4
batch_normalization_4
activation_4
scalar_multiplication



model_1
--------------------
input_8
periodic_padding_5
conv2d_transpose
cropping2d
batch_normalization_5
activation_5
periodic_padding_6
conv2d_transpose_1
cropping2d_1
batch_normalization_6
activation_6
periodic_padding_7
conv2d_transpose_2
cropping2d_2
batch_normalization_7
activation_7
periodic_padding_8
conv2d_transpose_3
cropping2d_3
batch_normalization_8
activation_8
periodic_padding_9
conv2d_transpose_4
cropping2d_4
batch_normalization_9
activation_9
scalar_multiplication_1



model_2
--------------------
input_9
model
model_1





In [15]:
import h5py

In [16]:
with h5py.File(wt_file, 'r') as f:
    counter = 0
    for key in f.keys():
        print(counter, key)
        counter += 1
    print(counter)

0 activation
1 activation_1
2 activation_2
3 activation_3
4 activation_4
5 activation_5
6 activation_6
7 activation_7
8 activation_8
9 activation_9
10 batch_normalization
11 batch_normalization_1
12 batch_normalization_2
13 batch_normalization_3
14 batch_normalization_4
15 batch_normalization_5
16 batch_normalization_6
17 batch_normalization_7
18 batch_normalization_8
19 batch_normalization_9
20 conv2d
21 conv2d_1
22 conv2d_2
23 conv2d_3
24 conv2d_4
25 conv2d_transpose
26 conv2d_transpose_1
27 conv2d_transpose_2
28 conv2d_transpose_3
29 conv2d_transpose_4
30 cropping2d
31 cropping2d_1
32 cropping2d_2
33 cropping2d_3
34 cropping2d_4
35 gaussian_noise
36 model
37 model_1
38 model_2
39 periodic_padding
40 periodic_padding_1
41 periodic_padding_2
42 periodic_padding_3
43 periodic_padding_4
44 periodic_padding_5
45 periodic_padding_6
46 periodic_padding_7
47 periodic_padding_8
48 periodic_padding_9
49 top_level_model_weights
50


In [17]:
with h5py.File(wt_file, 'r') as f:
    print(f['conv2d/conv2d/kernel:0'])

<HDF5 dataset "kernel:0": shape (5, 5, 2, 8), type "<f4">


In [18]:
with h5py.File(dir_name_ae+'/final_net/final_net_ae_weights.h5', 'r') as f:
    counter = 0
    for key in f.keys():
        print(counter, key)
        counter += 1
    print(counter)

0 activation
1 activation_1
2 activation_2
3 activation_3
4 activation_4
5 activation_5
6 activation_6
7 activation_7
8 activation_8
9 activation_9
10 batch_normalization
11 batch_normalization_1
12 batch_normalization_2
13 batch_normalization_3
14 batch_normalization_4
15 batch_normalization_5
16 batch_normalization_6
17 batch_normalization_7
18 batch_normalization_8
19 batch_normalization_9
20 conv2d
21 conv2d_1
22 conv2d_2
23 conv2d_3
24 conv2d_4
25 conv2d_transpose
26 conv2d_transpose_1
27 conv2d_transpose_2
28 conv2d_transpose_3
29 conv2d_transpose_4
30 cropping2d
31 cropping2d_1
32 cropping2d_2
33 cropping2d_3
34 cropping2d_4
35 gaussian_noise
36 model
37 model_1
38 model_2
39 periodic_padding
40 periodic_padding_1
41 periodic_padding_2
42 periodic_padding_3
43 periodic_padding_4
44 periodic_padding_5
45 periodic_padding_6
46 periodic_padding_7
47 periodic_padding_8
48 periodic_padding_9
49 scalar_multiplication
50 scalar_multiplication_1
51 top_level_model_weights
52


In [19]:
with h5py.File(dir_name_ae+'/final_net/final_net_ae_weights.h5', 'r') as f:
#     print(np.array(f['scalar_multiplication/scalar_multiplication/scalar_multiplier:0']))
    print(f['periodic_padding_9/periodic_padding_9/N_mat:0'])


<HDF5 dataset "N_mat:0": shape (24, 28), type "<f4">


In [20]:
with h5py.File(dir_name_all_ae+'/ae_038/final_net/final_net_ae_weights.h5', 'r') as f:
    counter = 0
    for key in f.keys():
        print(counter, key)
        counter += 1
    print(counter)

0 activation
1 activation_1
2 activation_2
3 activation_3
4 activation_4
5 activation_5
6 activation_6
7 activation_7
8 activation_8
9 activation_9
10 batch_normalization
11 batch_normalization_1
12 batch_normalization_2
13 batch_normalization_3
14 batch_normalization_4
15 batch_normalization_5
16 batch_normalization_6
17 batch_normalization_7
18 batch_normalization_8
19 batch_normalization_9
20 conv2d
21 conv2d_1
22 conv2d_2
23 conv2d_3
24 conv2d_4
25 conv2d_transpose
26 conv2d_transpose_1
27 conv2d_transpose_2
28 conv2d_transpose_3
29 conv2d_transpose_4
30 cropping2d
31 cropping2d_1
32 cropping2d_2
33 cropping2d_3
34 cropping2d_4
35 gaussian_noise
36 model
37 model_1
38 model_2
39 periodic_padding
40 periodic_padding_1
41 periodic_padding_2
42 periodic_padding_3
43 periodic_padding_4
44 periodic_padding_5
45 periodic_padding_6
46 periodic_padding_7
47 periodic_padding_8
48 periodic_padding_9
49 scalar_multiplication
50 scalar_multiplication_1
51 top_level_model_weights
52


In [21]:
with h5py.File(dir_name_all_ae+'/ae_038/final_net/final_net_ae_weights.h5', 'r') as f:
    print(np.array(f['scalar_multiplication/scalar_multiplication/scalar_multiplier:0']).shape)


(1,)
